In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark import SparkContext
from operator import add

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('prices.csv')



In [ ]:
dataRDD=df.rdd

dataRDD.take(5)



In [ ]:
#Select the total volumes and the totals for Apple
totals=dataRDD.map(lambda x:int(x[-1])).reduce(lambda x,y:x+y)
print(totals)



In [ ]:
appleRDD=dataRDD.filter(lambda x:x[0]=="AAPL")
appleRDD.cache()
print (appleRDD.count())

In [ ]:
appleRDD.map(lambda x:int(x[-1])).reduce(add)




In [ ]:
max_vals=dataRDD.map(lambda x: x[-1]).max()

print(max_vals)

In [ ]:
dataRDD.filter(lambda x: x[-1]==max_vals).collect()



In [ ]:
#Determine company, date and time of the session(s) with
#the highest variability (max delta between high and low)

def delta(x,y):
    return x-y

#deltaRDD=dataRDD.map(lambda x: (x[0],x[1],x[2],delta(float(x[-4]),float(x[-3]))))
deltaRDD=dataRDD.map(lambda x: (x[0:3],delta(float(x[-4]),float(x[-3]))))
deltaRDD.take(1)


In [ ]:
max_var_sessions=deltaRDD.map(lambda x: x[-1]).max()

max_var_sessionsRDD=deltaRDD.filter(lambda x: x[-1] == max_var_sessions)

max_var_sessionsRDD.collect()





In [ ]:
df.printSchema()



In [ ]:
#Select the total volumes and the totals for Apple
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType


df_data=df.withColumn("OPEN", df["OPEN"].cast("double")).\
           withColumn("HIGH", df["HIGH"].cast("double")).\
           withColumn("LOW", df["LOW"].cast("double")).\
           withColumn("CLOSE", df["CLOSE"].cast("double")).\
           withColumn("VOLUME ", df["VOLUME "].cast("int"))

df_data.printSchema()








In [ ]:
totals=df_data.groupBy().sum("volume ").collect()[0][0]
print(totals)



In [ ]:
df_apple=df_data.filter("SYMBOL == 'AAPL'")
df_apple.cache()

print (df_apple.count())



In [ ]:
df_apple.groupBy().sum("volume ").collect()[0][0]








In [ ]:
#Determine company, date and time of the session(s) with
#the highest variability (max delta between high and low)
from pyspark.sql.functions import udf

delta_udf=udf(delta,DoubleType())


df_delta=df_data.select("SYMBOL","DATE","TIME",delta_udf("HIGH","LOW"))\
                .withColumnRenamed("delta(HIGH, LOW)","DIFF_HIGH_LOW")
df_delta.show()



In [ ]:
max_var_sessions=df_delta.select("DIFF_HIGH_LOW").groupBy().max().collect()[0][0]

print (max_var_sessions)




In [ ]:

df_max_var_sessions=df_delta.filter( max_var_sessions == df_delta.DIFF_HIGH_LOW)


df_max_var_sessions.show()


